In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_json('dataset/no_pii_grievance.json')

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175784 entries, 0 to 175783
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   175784 non-null  object 
 1   CategoryV7            63622 non-null   float64
 2   DiaryDate             175784 non-null  object 
 3   UserCode              98097 non-null   float64
 4   closing_date          162365 non-null  object 
 5   dist_name             122808 non-null  object 
 6   org_code              175401 non-null  object 
 7   pincode               161502 non-null  object 
 8   recvd_date            175784 non-null  object 
 9   registration_no       175784 non-null  object 
 10  remarks_text          106904 non-null  object 
 11  resolution_date       106904 non-null  object 
 12  sex                   175784 non-null  object 
 13  state                 175521 non-null  object 
 14  subject_content_text  175784 non-null  object 
 15  v7_ta

In [28]:
data = pd.read_csv('dataset/Complaint Category.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19853 entries, 0 to 19852
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Code            19853 non-null  int64  
 1   Description     19853 non-null  object 
 2   OrgCode         19853 non-null  object 
 3   Parent          19762 non-null  float64
 4   Stage           19853 non-null  int64  
 5   MonitoringCode  16382 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 930.7+ KB


In [36]:
import pandas as pd
import numpy as np

df = pd.read_json('dataset/no_pii_grievance.json')
data = pd.read_csv('dataset/Complaint Category.csv')

# Create a dictionary to store parent-child relationships
parent_dict = dict(zip(data['Code'], data['Parent']))

# Create a dictionary to store code-description relationships
description_dict = dict(zip(data['Code'], data['Description']))

def find_root_node(code):
    try:
        current_code = code

        while current_code in parent_dict and not pd.isna(parent_dict[current_code]):
            current_code = parent_dict[current_code]

        return current_code
    except Exception as e:
        return f"Error: {e}"

def apply_root_category(row):
    if pd.isna(row['CategoryV7']):
        return np.nan
    else:
        root_node = find_root_node(row['CategoryV7'])
        return root_node if not isinstance(root_node, str) or not root_node.startswith("Error") else np.nan

def apply_root_category_description(row):
    if pd.isna(row['CategoryV7']):
        return np.nan
    else:
        root_description = description_dict.get(find_root_node(row['CategoryV7']), np.nan)
        return root_description if not isinstance(root_description, str) or not root_description.startswith("Error") else np.nan

# Apply the functions to the dataframe
df_subset = df.copy()
df_subset['root_category_name'] = df_subset.apply(apply_root_category_description, axis=1)
df_subset['root_category_code'] = df_subset.apply(apply_root_category, axis=1)
df_subset.head(10)

,_id,CategoryV7,DiaryDate,UserCode,closing_date,dist_name,org_code,pincode,recvd_date,registration_no,remarks_text,resolution_date,sex,state,subject_content_text,v7_target,root_category_name,root_category_code
0,MORLY/E/2023/0000001,11578.0,{'$date': '2023-01-01T00:00:19.977Z'},110124.0,{'$date': '2023-01-04T00:00:00Z'},North 24 Parganas,MORLY,700130,{'$date': '2023-01-01T00:00:19.977Z'},MORLY/E/2023/0000001,"As per railway record, there is no authoriz...",{'$date': '2023-01-04T00:00:00Z'},M,WB,"Railways, ( Railway Board) >> Miscellaneous\r\...",No,Railway,2565.0
1,GOVUP/E/2023/0000001,NaN,{'$date': '2023-01-01T00:01:29.78Z'},45427.0,{'$date': '2023-01-24T00:00:00Z'},NaN,GOVUP,203001,{'$date': '2023-01-01T00:01:28.567Z'},GOVUP/E/2023/0000001,NaN,NaN,M,UP,XAXPX/X/X0X2X4X0X0\tREGARDING CBCID INSPECTION...,NaN,NaN,NaN
2,MOLBR/E/2023/0000001,2369.0,{'$date': '2023-01-01T00:01:45.593Z'},1356254.0,{'$date': '2023-01-12T00:00:00Z'},Hyderabad,MOLBR,500023,{'$date': '2023-01-01T00:01:45.593Z'},MOLBR/E/2023/0000001,"Sir/Madam, With reference to Grievance no. XO...",{'$date': '2023-01-12T00:00:00Z'},M,TG,Labour and Employment >> PF Withdrawal >> Othe...,No,Labour and Employment,2173.0
3,MOLBR/E/2023/0000002,2379.0,{'$date': '2023-01-01T00:02:07.247Z'},1092136.0,{'$date': '2023-01-06T00:00:00Z'},Nagpur,MOLBR,440001,{'$date': '2023-01-01T00:02:07.247Z'},MOLBR/E/2023/0000002,Please submit establishment clarification let...,{'$date': '2023-01-06T00:00:00Z'},M,MH,Labour and Employment >> Pension >> Others\r\n...,No,Labour and Employment,2173.0
4,GOVUP/E/2023/0000002,NaN,{'$date': '2023-01-01T00:02:25.663Z'},45427.0,{'$date': '2023-01-24T00:00:00Z'},NaN,GOVUP,203001,{'$date': '2023-01-01T00:02:24.913Z'},GOVUP/E/2023/0000002,NaN,NaN,M,UP,XAXPX/X/X0X2X4X0X8\tREGARDING CBCID INSPECTION...,NaN,NaN,NaN
5,GOVUP/E/2023/0000003,NaN,{'$date': '2023-01-01T00:03:02.52Z'},45427.0,{'$date': '2023-01-24T00:00:00Z'},NaN,GOVUP,203001,{'$date': '2023-01-01T00:03:01.79Z'},GOVUP/E/2023/0000003,NaN,NaN,M,UP,XAXPX/X/X0X2X4X0X7\tREGARDING CBCID INSPECTION...,NaN,NaN,NaN
6,MODEF/E/2023/0000001,20493.0,{'$date': '2023-01-01T00:04:02.5Z'},408844.0,{'$date': '2023-01-03T00:00:00Z'},Jammu,MODEF,181205,{'$date': '2023-01-01T00:04:02.5Z'},MODEF/E/2023/0000001,Son is eligible for dependent CSD Smart Card ...,{'$date': '2023-01-03T00:00:00Z'},F,JK,Defence >> Canteen Stores Depot related >> Non...,Yes,Department of Defence,6300.0
7,DEPOJ/E/2023/0000002,5270.0,{'$date': '2023-01-01T00:04:11.57Z'},951216.0,{'$date': '2023-01-04T00:00:00Z'},Kanyakumari,DEPOJ,629702,{'$date': '2023-01-01T00:04:11.57Z'},DEPOJ/E/2023/0000002,S.P/kkLR.No.P3//22 Dated 04.01.2023 மனுதாரர் ...,{'$date': '2023-01-04T00:00:00Z'},M,TN,Justice >> Others\r\n-----------------------\r...,Yes,Department of Justice,5256.0
8,MEAPD/E/2023/0000001,1441.0,{'$date': '2023-01-01T00:04:30.55Z'},711170.0,{'$date': '2023-01-09T00:00:00Z'},Bhupalpally (Jayashankar),MEAPD,505184,{'$date': '2023-01-01T00:04:30.55Z'},MEAPD/E/2023/0000001,"As per the HCI, Wellington, the service sough ...",{'$date': '2023-01-09T00:00:00Z'},M,TG,External Affairs >> Others\r\n----------------...,Yes,External Affairs,1221.0
9,DORLD/E/2023/0000001,5073.0,{'$date': '2023-01-01T00:05:15.5Z'},20318.0,None,Kandhamal,DORLD,762022,{'$date': '2023-01-01T00:05:15.5Z'},DORLD/E/2023/0000001,ग्रामीण सड़क राज्य सरकार का विषय है और राज्य ...,{'$date': '2023-01-31T00:00:00Z'},M,OR,Rural Development >> Pradhan Mantri Gram Sadak...,Yes,Rural Development,4976.0


In [37]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175784 entries, 0 to 175783
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   175784 non-null  object 
 1   CategoryV7            63622 non-null   float64
 2   DiaryDate             175784 non-null  object 
 3   UserCode              98097 non-null   float64
 4   closing_date          162365 non-null  object 
 5   dist_name             122808 non-null  object 
 6   org_code              175401 non-null  object 
 7   pincode               161502 non-null  object 
 8   recvd_date            175784 non-null  object 
 9   registration_no       175784 non-null  object 
 10  remarks_text          106904 non-null  object 
 11  resolution_date       106904 non-null  object 
 12  sex                   175784 non-null  object 
 13  state                 175521 non-null  object 
 14  subject_content_text  175784 non-null  object 
 15  v7_ta

In [72]:
columns_to_select = ['_id', 'CategoryV7', 'remarks_text', 'subject_content_text', 'root_category_name', 'root_category_code']
existing_columns = [col for col in columns_to_select if col in df_subset.columns]

# Create a new DataFrame with non-null values of CategoryV7 and selected columns
new_dataset = df_subset.dropna(subset=['CategoryV7'], how='any', inplace=False)[existing_columns]
json_filename = 'test/root_category_dataset.json'
new_dataset.to_json(json_filename, orient='records', lines=False, default_handler=str, indent=2)

In [79]:
data = pd.read_csv('dataset/Complaint Category.csv')

def find_root_node_with_descriptions(data, code):
    try:
        current_code = code
        row = data[data['Code'] == current_code].iloc[0]
        hierarchy = [f"{row['Description']}"]
        hierarchy_code_array = [row['Code']]
        total_stages = 0

        while not pd.isna(row['Parent']):
            current_code = row['Parent']
            row = data[data['Code'] == current_code].iloc[0]
            # Insert at the beginning to maintain order from root to given code
            hierarchy.insert(0, f"{row['Description']}")
            hierarchy_code_array.insert(0, row['Code'])  # Insert at the beginning to maintain order from root
            total_stages += 1

        return row['Code'], total_stages + 1, hierarchy, hierarchy_code_array
    except Exception as e:
        return None, None, [], [], f"Error: {e}"

def add_hierarchy_info(row, data):
    root_code, total_stages, hierarchy, hierarchy_code_array = find_root_node_with_descriptions(data, row['CategoryV7'])
    row['total_no_of_stages'] = total_stages
    row['hierarchy_order'] = hierarchy
    row['category_hierarchy_code'] = hierarchy_code_array
    return row

new_df = new_dataset.apply(lambda row: add_hierarchy_info(row, data), axis=1)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63622 entries, 0 to 175783
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      63622 non-null  object 
 1   CategoryV7               63622 non-null  float64
 2   remarks_text             53566 non-null  object 
 3   subject_content_text     63622 non-null  object 
 4   root_category_name       63622 non-null  object 
 5   root_category_code       63622 non-null  float64
 6   total_no_of_stages       63622 non-null  int64  
 7   hierarchy_order          63622 non-null  object 
 8   category_hierarchy_code  63622 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 6.9+ MB


In [80]:
# find the null in remark text and print that row
new_df[new_df['remarks_text'].isnull()]['_id']

12        MINHA/E/2023/0000002
21        MINHA/E/2023/0000003
75        DHLTH/E/2023/0000002
76        DLGLA/E/2023/0000001
82        DOSEL/E/2023/0000001
                  ...         
175760    DOAAC/E/2023/0006498
175764    CBODT/E/2023/0006286
175767    CBODT/E/2023/0006288
175769    DHLTH/E/2023/0001299
175783    MODEF/E/2023/0000652
Name: _id, Length: 10056, dtype: object

In [60]:
new_df.head(100).to_json('sample/sample_dataset.json', orient='records', lines=False, default_handler=str, indent=2)